# Chemistry example: Cement

This example is about chemical components. The idea is to take input from a natural language text that contains descriptions about chemical components. With Natural Language Processing the relevant component names shall be extracted and the corresponding components shall be looked up in the Wikidata knowledge graph.

## Example
Input as natural text: "Polylactic acid (PLA) and Acrylonitrile butadiene styrene (ABS) are 3D printing filaments."
Extraction of component names:

* PLA - Polylactic acid
* ABS - Acrylonitrile butadiene styrene

Lookup in Wikidata as:
* [PLA - Q413769](https://www.wikidata.org/wiki/Q413769) (C₃H₆O₃)x
* [ABS - Q143496](https://www.wikidata.org/wiki/Q143496) C₁₅H₁₇N

# Install dependencies

In [19]:
import sys
def installModule(projectName:str, moduleName:str=None):
    '''Installs and loads the given module if not already installed'''
    if moduleName is None:
        moduleName=projectName
    if moduleName not in sys.modules:
        !python -m pip install --no-input $projectName
        print(f'{projectName} installed')
    else:
        print(f'{projectName} found')
    %reload_ext $moduleName

installModule('jupyter-xml')
installModule('jupyter-rdfify')
installModule('SPARQLWrapper')
installModule('tabulate')
installModule('spacy')

jupyter-xml found


Defaulting to user installation because normal site-packages is not writeable
jupyter-rdfify installed


ImportError: cannot import name '_RDFNamespace' from 'rdflib.namespace' (/Users/wf/Library/Python/3.9/lib/python/site-packages/rdflib/namespace/__init__.py)

# Download Models

In [11]:
!python -m spacy download en_core_web_sm    # efficent
!python -m spacy download en_core_web_trf   # accurate


✘ No compatible model found for 'en_core_web_trf' (spaCy v2.3.7).



# Chemistry Example Wikidata Query
see http://wiki.bitplan.com/index.php/PyLoDStorage#15_Random_substances_with_CAS_number

## Extract text from website

In [9]:
from newspaper import Article
url="https://www.engr.psu.edu/ce/courses/ce584/concrete/library/construction/curing/Composition%20of%20cement.htm"
article = Article(url)
article.download()
article.parse()
text=article.text

# NLP with spacy
Try to identify [chemical compounds](https://www.wikidata.org/wiki/Q11173)

In [10]:
import spacy
from tabulate import tabulate
# Load English tokenizer, tagger, parser and NER
nlp = spacy.load('en_core_web_trf')
doc = nlp(text)
nouns=[chunk.text for chunk in doc.noun_chunks]

print(f"Found nouns:\n {nouns}")

foundEntities=[{"Text":entity.text, "Entity Tag":entity.label_} for entity in doc.ents]
print(tabulate(foundEntities, headers="keys"))

OSError: [E050] Can't find model 'en_core_web_trf'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

# Query wikidata for mentioned Chemical Compounds
The NER (Named Entity Recognition) seems not to detect the chemical compounds. Thus we use the found nouns to query wikidata for the referenced compounds

<div class="alert alert-block alert-warning">
<b>ToDo:</b> Optimize query some components that are referenced do not have all queried properties. Making the properties OPTIONAL results in an timeout.
</div>

In [13]:
from SPARQLWrapper import SPARQLWrapper, JSON, CSV
from tabulate import tabulate

queryValueFormatTags='%s,\n'*((len(nouns)//100)) + '%s'
q = """
SELECT DISTINCT ?substance ?substanceLabel ?formula ?structure ?CAS
WHERE { 
  ?substance wdt:P31 wd:Q11173.
  ?substance wdt:P231 ?CAS.
  ?substance wdt:P274 ?formula.
  ?substance wdt:P117  ?structure.
  ?substance rdfs:label ?substanceLabel
  FILTER(str(?substanceLabel) in ( %s ))
}
LIMIT 50

""" % queryValueFormatTags
values=tuple([', '.join([f'"{noun.strip()}"' for noun in nouns[n:n+100]]) for n in range((len(nouns)//100)+1)])
q = q % values
sparql = SPARQLWrapper("http://query.wikidata.org/sparql")
sparql.setQuery(q)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
table = [[result[column]["value"] for column in result] for result in results["results"]["bindings"]]
print(tabulate(table))

NameError: name 'nouns' is not defined

<div class="alert alert-block alert-warning">
<b>ToDo:</b> Decide which SPARQL query framework sould be used and simplfy the access and interface for the usage in jupyter notebooks
</div>